<br>

# Setores

- https://www.tjsp.jus.br/QuemSomos/QuemSomos/RegioesAdministrativasJudiciarias

<br>

Michel Metran\
Data: 17.06.2024\
Atualizado em: 17.06.2024


In [ ]:
#!pip3 install requests-ip-rotator

In [ ]:
#!pip3 install traquitanas

In [ ]:
import concurrent
import os
import re
from typing import Literal

import pandas as pd
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from open_geodata import geo
from requests_ip_rotator import ApiGateway
from selenium.webdriver.common.by import By

from sp_tjsp_divadmin.my_driver import Driver
from sp_tjsp_divadmin.my_functions import (
    find_text_between_parenthesis,
    keep_numbers,
)
from sp_tjsp_divadmin.my_paths import (
    adds_path,
    driver_path,
    logs_path,
    output_path_tab,
)

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
# from selenium.webdriver.firefox.service import Service as FirefoxService
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.support.wait import WebDriverWait
# from traquitanas.scrapping import adds, gecko

In [ ]:
load_dotenv()

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

<br>

---

## Unidades


In [ ]:
# Preciso listar Imóveis para obter o código
df_unidade = pd.read_csv(output_path_tab / 'Unidades.csv')

# ddd
list_unidades = list(df_unidade['unidade'])
list_unidades[:5]

<br>

---

## Setores


In [ ]:
def get_id_setor(setor) -> pd.DataFrame:
    """
    Obtem o código (ou ID) dos Setores do TJSP

    :param unidade: _description_
    :return: tabela com os valores
    """
    # Get Data
    r = requests.post(
        'https://www.tjsp.jus.br/AutoComplete/ListarSetores',
        json={'texto': setor},
    )

    # Se é lista vazia, ignora
    if r.json() == 'listaVazia':
        pass

    # Se não é, registra!
    else:
        df = pd.DataFrame(r.json())
        df = df.rename(
            mapper={
                'Codigo': 'id_setor',
                'Descricao': 'setor',
            },
            axis='columns',
        )
        df[['setor', 'unidade']] = df['setor'].str.split('|', expand=True)
        df = df.sort_values(by='id_setor', ascending=True)
        return df

In [ ]:
#
df = get_id_setor(setor=list_unidades[0])
df.to_clipboard()
df

<br>

-----

## dddd

In [ ]:
def get_detalhes_setor(id_setor):
    """
    _summary_
    Uma vez com o Código do Imóvel, consigo obter detalhes

    :param id_setor: _description_
    :type id_setor: _type_
    :return: _description_
    :rtype: _type_
    """
    r = requests.post(
        'https://www.tjsp.jus.br/ListaTelefonica/RetornarResultadoBusca',
        json={'parmsEntrada': id_setor, 'codigoTipoBusca': 3},
    )

    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [ ]:
def get_detalhes_setor_soup(id_setor, soup):
    """
    _summary_

    :param soup: _description_
    :return: _description_
    """

    try:
        nome_setor = soup.find(
            name='h2', attrs={'class': 'setortitle'}
        ).text.strip()
    except:
        nome_setor = 'Sem nome'

    try:
        dados_gerais = (
            soup.find(
                'dt',
                string=re.compile('.*Dados Gerais.*', flags=re.DOTALL),
            )
            .parent.find(name='dd')
            .text.strip()
        )
    except:
        dados_gerais = 'Não Encontrado'

    try:
        telefone = (
            soup.find(
                'dt',
                string=re.compile('.*Telefone Principal.*', flags=re.DOTALL),
            )
            .parent.find(name='dd')
            .find(name='span')
            .text.strip()
        )
    except:
        telefone = 'Não Encontrado'

    try:
        fax = (
            soup.find(
                'dt',
                string=re.compile('.*Fax.*', flags=re.DOTALL),
            )
            .parent.find(name='dd')
            .find(name='span')
            .text.strip()
        )
    except:
        fax = 'Não Encontrado'

    try:
        email_setor = (
            soup.find(
                'dt',
                string=re.compile('.*Email Setor.*', flags=re.DOTALL),
            )
            .parent.find(name='dd')
            .find(name='span')
            .text.strip()
        )
    except:
        email_setor = 'Não Encontrado'

    try:
        id_unidade = int(
            soup.find(name='input', attrs={'id': 'hdCodigoImovel'}).get('value')
        )
    except:
        id_unidade = 'Não Encontrado'

    return {
        'id_setor': id_setor,
        'setor': nome_setor,
        'dados_gerais': dados_gerais,
        'telefone': telefone,
        'fax': fax,
        'email': email_setor,
        'id_unidade': id_unidade,
    }

In [ ]:
def combo(id_setor):

    soup = get_detalhes_setor(id_setor=id_setor)
    # print(soup.prettify())
    return get_detalhes_setor_soup(id_setor=id_setor, soup=soup)

In [ ]:
id_setor = 3534

combo(id_setor=id_setor)

In [ ]:
# Cria Gateway
gateway = ApiGateway(
    site='https://www.tjsp.jus.br',
    access_key_id=aws_access_key_id,
    access_key_secret=aws_secret_access_key,
    regions=['sa-east-1'],
    verbose=True,
)
gateway.pool_connections = 10
gateway.pool_maxsize = 10
gateway.start()

# Cria Session
session = requests.Session()
session.mount(prefix='https://www.tjsp.jus.br', adapter=gateway)

In [ ]:
# Parameters
MAX_THREADS = 4
list_dfs = []
list_futures = []

# Paralelo
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    # Cria Lista de Tarefas
    for i in range(1, 6000):
        futures = executor.submit(combo, i)
        list_futures.append(futures)

    # Executa Lista de Tarefas
    for future in concurrent.futures.as_completed(list_futures):
        print(future.result())
        list_dfs.append(future.result())

# Encerra o worker
gateway.shutdown()

In [ ]:
# Junta
df_setores = pd.DataFrame(list_dfs)
df_setores = df_setores.drop_duplicates()

# Filtra o que é de interesse
mask = (
    (df_setores['setor'] == 'Sem nome')
    & (df_setores['dados_gerais'] == 'Não Encontrado')
    & (df_setores['telefone'] == 'Não Encontrado')
    & (df_setores['fax'] == 'Não Encontrado')
    & (df_setores['email'] == 'Não Encontrado')
    & (df_setores['id_unidade'] == 'Não Encontrado')
)
df_setores = df_setores[~mask]


# Remove ; do fim da coluna
df_setores['telefone'] = df_setores['telefone'].str.replace(
    r'\;$', '', regex=True
)
df_setores['fax'] = df_setores['fax'].str.replace(r'\;$', '', regex=True)
df_setores['email'] = df_setores['email'].str.replace(r'\;$', '', regex=True)


# Aplica strip em todo o dataframe
df_setores = df_setores.map(lambda x: x.strip() if isinstance(x, str) else x)

# Reordena
df_setores = df_setores.sort_values(by='id_setor', ascending=True)
df_setores = df_setores.reset_index(drop=True)

# Converte para inteiro
df_setores['id_unidade'] = df_setores['id_unidade'].astype(int)

# Resultados
df_setores.info()
display(df_setores.head())
display(df_setores.tail())

In [ ]:
filename = 'Setores'

# Salva
df_setores.to_csv(output_path_tab / f'{filename}.csv', index=False)
df_setores.to_excel(
    output_path_tab / f'{filename}.xlsx', sheet_name=f'{filename}', index=False
)